In [1]:
# import packages we need to use
import scipy.stats
import math
import pandas as pd
import datetime

In [2]:
#link = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
link = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
# You have to download the csv file from the link above, 
#  save it under *.csv, then open it with Excel and save it again (without doing anything else)

#file = 'C:\\data\\UScovidCountiesCurrent_Confirmed.csv'
#file = 'C:\\data\\UScovidCountiesCurrent_Deaths.csv'
#v =  pd.read_csv(file,sep=',',header=0)
v =  pd.read_csv(link,sep=',',header=0)

In [3]:
v.shape

(3261, 160)

In [4]:
#Cleaning
v = v[v.iso3 =='USA'];
v = v.drop(['iso2', 'iso3', 'code3', 'Country_Region'], axis=1);
v = v[v.Lat > 0.0] #getting rid of some "unassigned" locations

In [5]:
dateList = v.columns[7:v.columns.size+1]

In [6]:
months = {'1' : 'Jan',
          '2' : 'Feb',
          '3' : 'Mar',
          '4' : 'Apr',
          '5' : 'May',
          '6' : 'Jun',
          '7' : 'Jul',
          '8' : 'Aug',
          '9' : 'Sep',
          '10' : 'Oct',
          '11' : 'Nov',
          '12' : 'Dec'
}

In [7]:
def niceDate(slashesDate,letterMonth=True):
    # if letterMonth=True, converting 3/15/20 to Mar-15-2020 and 3/7/20 to Mar-07-2020
    # else, converting 3/15/20 to 03-15-2020 and 3/7/20 to 03-07-2020
    global months
    loc = 0 #<-starting point is location zero
    
    # create a 3 letter month
    # we need to detect if slashesDate starts with one or two digits
    if slashesDate[1] == '/':
        if letterMonth:
            nice = months[slashesDate[0]]
        else:
            nice = '0' + slashesDate[0]
        loc = 2
    else:
        if letterMonth:
            nice = months[slashesDate[0:2]]
        else:
            nice = slashesDate[0:2]
        loc = 3
    
    nice +='-' #adding the dash
    
    # Next we need to figure out the day
    if slashesDate[loc+1] == '/':
        nice += '0' + slashesDate[loc]
        loc +=2
    else:
        nice += slashesDate[loc:loc+2] 
        loc +=3
    
    #next we just add the 2020 year
    nice += '-20' + slashesDate[loc:loc+2]
    return nice

        

In [8]:
niceDateList = []
for d in dateList:
    niceDateList.append(niceDate(d,False))

In [9]:

print('First Date in data set: ',niceDateList[0])
print('Last Date in data set: ',niceDateList[-1])

First Date in data set:  01-22-2020
Last Date in data set:  06-18-2020


In [10]:
from ipyleaflet import Map, Marker, MarkerCluster, Icon, Heatmap #, FullScreenControl

In [11]:
# USA center:
center=(38.2,-92.0)

# create the map:
m=Map(center=center,zoom=4)
#m.add_control(FullScreenControl())

# import the GeoJson package
from ipyleaflet import GeoJSON
import json

#create a layer with state borders
with open("c:/data/us_states.json", 'r', encoding='utf-8') as f:
  data = json.load(f)

# add the states borders layer to the map
geo_json = GeoJSON(data=data, style = {'color': 'green', 'opacity':0.4, 'weight':1,  'fillOpacity':0.01})
m.add_layer(geo_json)

In [12]:
currentDate = dateList[0] 

In [13]:
def heat(df,col):
    #Mean,Var = 50,2500
    loc = [];
    for date,row in df.iterrows():
        #temp = scipy.stats.norm(Mean, Var).cdf(row.absolute_confirmed/1000)
        #loc.append([row.location_lat,row.location_long,temp])
        #loc.append([row.location_lat,row.location_long,row.absolute_confirmed])
        loc.append([row.iloc[4],row.iloc[5], row.iloc[col]])
    return loc


In [14]:
col = v.columns.size-1
loc = heat(v,col)

In [15]:
heatmap = Heatmap(
    locations=loc,
    radius=20,
    gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}
)

m.add_layer(heatmap);

In [16]:
display(m)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [17]:
print(v.columns[col])

6/18/20
